# Modular CerberusTS Experimentation

In [1]:
import os
os.chdir("..")

In [2]:
from cerberus_ts import Cerberus, train_cerberus
from cerberus_ts import prepare_timeseries_data, generate_predictions, denormalize_response
import pandas as pd

c:\Users\johnm\AppData\Local\Programs\Python\Python38\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


ImportError: cannot import name 'denormalize_response' from 'cerberus_ts' (c:\Users\johnm\Documents\GitHub\cerberusPy\cerberus_ts\__init__.py)

In [ ]:
df = pd.read_csv(r"data/jena_climate_2009_2016.csv",
                parse_dates=['Date Time'], 
                index_col=['Date Time'])
df.index = pd.to_datetime(df.index, format='%d.%m.%Y %H:%M:%S')
df = df.iloc[:5000,:]

In [ ]:
thresholds = {
    'call': 1,
    'response': 1,
    'context_0': 1,
    'context_1': 1,
    'context_2': 1
}

sizes = {
    'call': 24,
    'response': 8,
    'context_0': 24,
    'context_1': 12,
    'context_2': 6
}

window_timesteps = {
    'call': '10T',
    'response': '10T',
    'context_0': '1H',
    'context_1': '2H',
    'context_2': '6H'
}

window_sizes = {
    'call': 24,
    'response': 8,
    'context_0': 24,
    'context_1': 12,
    'context_2': 6
}

feature_indexes = {
    'call': range(0,14),
    'response': [0, 1, 4],
    'context_0': range(0,14),
    'context_1': range(0,14),
    'context_2': range(0,14)   
}

In [ ]:
prepared_dataloaders, sliced_data, max_change_dfs = prepare_timeseries_data(df,
                        sizes,
                        thresholds,
                        feature_indexes,
                        window_timesteps,
                        train_len = 20_000,
                        feature_range = [0, 1])

In [ ]:
model = Cerberus(sizes, feature_indexes)

In [ ]:
model = train_cerberus(model, prepared_dataloaders, num_epochs = 30)

In [ ]:
sel_indx = 4000
# Move model over to cpu
model.to("cpu")

# Set the model to evaluation mode
model.eval()
selected_data = {key: value[sel_indx:sel_indx+1,:] for key, value in sliced_data.items()}
responses_generated = generate_predictions(model,selected_data)

In [ ]:
responses_generated

In [ ]:
initial_value = selected_data['last_known'][0][0][feature_indexes['response']]
max_change_df = max_change_dfs['response']

denormalized_response = denormalize_response(responses_generated, max_change_df,initial_value)
print(denormalized_response)